# 第2章: UNIXコマンドの基礎

In [14]:
datapath = "../../problem/data/popular-names.txt"

### 10. 行数のカウント

In [15]:
with open(datapath) as f:
    print(sum(1 for line in f))

2740


In [16]:
%%bash
wc -l ../../problem/data/popular-names.txt

2740 ../../problem/data/popular-names.txt


### 11. タブをスペースに置換

In [17]:
with open(datapath) as f:
    l = [line.replace("\t", " ").rstrip() for line in f]
l[:10]

['Mary F 7065 1880',
 'Anna F 2604 1880',
 'Emma F 2003 1880',
 'Elizabeth F 1939 1880',
 'Minnie F 1746 1880',
 'Margaret F 1578 1880',
 'Ida F 1472 1880',
 'Alice F 1414 1880',
 'Bertha F 1320 1880',
 'Sarah F 1288 1880']

In [18]:
%%bash
sed "s/\t/ /g" ../../problem/data/popular-names.txt | head
echo -----------------
tr '\t' ' ' < ../../problem/data/popular-names.txt | head
echo -----------------
expand -t 1 ../../problem/data/popular-names.txt | head

Mary F 7065 1880
Anna F 2604 1880
Emma F 2003 1880
Elizabeth F 1939 1880
Minnie F 1746 1880
Margaret F 1578 1880
Ida F 1472 1880
Alice F 1414 1880
Bertha F 1320 1880
Sarah F 1288 1880
-----------------
Mary F 7065 1880
Anna F 2604 1880
Emma F 2003 1880
Elizabeth F 1939 1880
Minnie F 1746 1880
Margaret F 1578 1880
Ida F 1472 1880
Alice F 1414 1880
Bertha F 1320 1880
Sarah F 1288 1880
-----------------
Mary F 7065 1880
Anna F 2604 1880
Emma F 2003 1880
Elizabeth F 1939 1880
Minnie F 1746 1880
Margaret F 1578 1880
Ida F 1472 1880
Alice F 1414 1880
Bertha F 1320 1880
Sarah F 1288 1880


### 12. 1列目をcol1.txtに，2列目をcol2.txtに保存

In [19]:
col1 = ""
col2 = ""
with open(datapath) as f:
    for line in f:
        splits = line.split()
        col1 += "".join([splits[0], "\n"])
        col2 += "".join([splits[1], "\n"])
        
with open("col1.txt", "w") as f1, open("col2.txt", "w") as f2:
    f1.write(col1)
    f2.write(col2)
    

import pandas as pd

df = pd.read_table('../../problem/data/popular-names.txt', header=None)
l = list(df[0].head(10))
l

['Mary',
 'Anna',
 'Emma',
 'Elizabeth',
 'Minnie',
 'Margaret',
 'Ida',
 'Alice',
 'Bertha',
 'Sarah']

In [20]:
%%bash
cut -f 1 ../../problem/data/popular-names.txt | head

Mary
Anna
Emma
Elizabeth
Minnie
Margaret
Ida
Alice
Bertha
Sarah


### 13. col1.txtとcol2.txtをマージ

In [21]:
with open("col1.txt") as f1, open("col2.txt") as f2:
    print(("\t".join([line1.strip(),line2]) for line1,line2 in zip(f1, f2)))

<generator object <genexpr> at 0x7f0070928f68>


In [37]:
t = ""
with open("col1.txt") as f1, open("col2.txt") as f2:
    for line1,line2 in zip(f1, f2):
        t += "\t".join([line1.strip(),line2])
        
with open("merge.txt", "w")as f:
    f.write(t)


l = list(df[0].str.cat(df[1], sep='\t').head(10))
print(*l, sep='\n')

Mary	F
Anna	F
Emma	F
Elizabeth	F
Minnie	F
Margaret	F
Ida	F
Alice	F
Bertha	F
Sarah	F


In [23]:
%%bash
paste col1.txt col2.txt | head

Mary	F
Anna	F
Emma	F
Elizabeth	F
Minnie	F
Margaret	F
Ida	F
Alice	F
Bertha	F
Sarah	F


### 14. 先頭からN行を出力

In [24]:
# input [N]
arg = input("input a number: ")
with open(datapath) as f:
    for _ in range(int(arg)):
        print(f.readline(), end="")

Mary	F	7065	1880
Anna	F	2604	1880
Emma	F	2003	1880


In [25]:
%%bash
head -3 ../../problem/data/popular-names.txt

Mary	F	7065	1880
Anna	F	2604	1880
Emma	F	2003	1880


### 15. 末尾のN行を出力

In [26]:
import linecache
arg = int(input("input a number: "))

with open(datapath) as f:
    size = sum(1 for _ in f)
startline = size - arg + 1
for i in range(startline, size + 1):
    print(linecache.getline(datapath, i), end="")
linecache.clearcache()

print('----------------------------')

import collections
with open(datapath) as f:
    deque = collections.deque(f, arg)
for line in deque:
    print(line, end='')

Michael	M	13998	2016
Elijah	M	13764	2016
Ethan	M	13758	2016
----------------------------
Michael	M	13998	2016
Elijah	M	13764	2016
Ethan	M	13758	2016


In [27]:
%%bash
tail -3 ../../problem/data/popular-names.txt

Michael	M	13998	2016
Elijah	M	13764	2016
Ethan	M	13758	2016


### 16. ファイルをN分割する

In [28]:
arg = input("input a number: ")
n = int(arg)
with open(datapath) as f:
    size = sum(1 for line in f)
    f.seek(0)
    length = int(size / n)
    mod = int(size % n)
    for i in range(n):
        splited = ""
        for _ in range(length):
            splited += f.readline()
        if(i < mod):
            splited += f.readline()
        with open("x{}.txt".format(i), "w") as wf:
            print(splited.count('\n'))
            wf.write(splited)

914
913
913


In [29]:
%%bash
split -n l/3 ../../problem/data/popular-names.txt
wc -l xaa
wc -l xab
wc -l xac

943 xaa
910 xab
887 xac


### 17. １列目の文字列の異なり
> 1列目の文字列の種類（異なる文字列の集合）を求めよ．確認にはsort, uniqコマンドを用いよ．

In [40]:
with open("col1.txt") as f:
    names = sorted({line.rstrip() for line in f})
    
print(*names[:10], sep="\n")

Abigail
Aiden
Alexander
Alexis
Alice
Amanda
Amy
Andrew
Angela
Anna


In [31]:
%%bash
sort col1.txt | uniq | head

Abigail
Aiden
Alexander
Alexis
Alice
Amanda
Amy
Andrew
Angela
Anna


### 18. 各行を3コラム目の数値の降順にソート
> 各行を3コラム目の数値の逆順で整列せよ（注意: 各行の内容は変更せずに並び替えよ）．確認にはsortコマンドを用いよ（この問題はコマンドで実行した時の結果と合わなくてもよい）．

In [46]:
with open(datapath) as f:
    sorted_lines = sorted(f, key=lambda x: int(x.split()[2]), reverse=True)
    
print(*sorted_lines[:10])

Linda	F	99685	1947
 Linda	F	96210	1948
 James	M	94762	1947
 Michael	M	92716	1957
 Robert	M	91641	1947
 Linda	F	91013	1949
 Michael	M	90620	1956
 Michael	M	90512	1958
 James	M	88584	1948
 Michael	M	88525	1954



In [33]:
%%bash
sort -nrk 3 ../../problem/data/popular-names.txt | head

Linda	F	99685	1947
Linda	F	96210	1948
James	M	94762	1947
Michael	M	92716	1957
Robert	M	91641	1947
Linda	F	91013	1949
Michael	M	90620	1956
Michael	M	90512	1958
James	M	88584	1948
Michael	M	88525	1954


### 19. 各行の1コラム目の文字列の出現頻度を求め，出現頻度の高い順に並べる
> 各行の1列目の文字列の出現頻度を求め，その高い順に並べて表示せよ．確認にはcut, uniq, sortコマンドを用いよ．

In [34]:
from collections import Counter

In [48]:
with open("col1.txt") as f:
    counter = Counter(line.rstrip() for line in f)
for w, n in counter.most_common()[:10]:
    print(n, w)
counter.most_common(10)

116 James
109 William
108 Robert
108 John
92 Mary
75 Charles
74 Michael
73 Elizabeth
71 Joseph
60 Margaret


[('James', 116),
 ('William', 109),
 ('Robert', 108),
 ('John', 108),
 ('Mary', 92),
 ('Charles', 75),
 ('Michael', 74),
 ('Elizabeth', 73),
 ('Joseph', 71),
 ('Margaret', 60)]

In [36]:
%%bash
sort col1.txt | uniq -c | sort -nr | head

    116 James
    109 William
    108 Robert
    108 John
     92 Mary
     75 Charles
     74 Michael
     73 Elizabeth
     71 Joseph
     60 Margaret
